# 中国 Shibor 3M 利率互换定盘曲线
-------------

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import datetime as dt
from QuantLib import *
plt.style.use('fivethirtyeight')

## 市场数据
-------------------

我们使用2017年6月30公布的利率互换定盘数据：

| 标准期限 | 报买(%) | 均值(%) | 报卖(%) |  |
|----------|---------|---------|---------|---|
| 6M | 4.4696 | 4.5128 | 4.5560 |  |
| 9M | 4.4100 | 4.4413 | 4.4725 |  |
| 1Y | 4.4100 | 4.4288 | 4.4475 |  |
| 2Y | 4.3900 | 4.4000 | 4.4100 |  |
| 3Y | 4.3900 | 4.4125 | 4.4350 |  |
| 4Y | 4.4050 | 4.4263 | 4.4475 |  |
| 5Y | 4.4100 | 4.4350 | 4.4600 |  |
| 7Y | 4.3954 | 4.4840 | 4.5725 |  |
| 10Y | 4.4863 | 4.5915 | 4.6967 |  |

In [ ]:
bid_rates = [0.044696, 0.0441, 0.0441, 0.0439, 0.0439, 0.04405, 0.0441, 0.043954, 0.044863]
mid_rates = [0.045128, 0.044413, 0.044288, 0.044, 0.044125, 0.044263, 0.04435, 0.04484, 0.045915]
offer_rates = [0.04556, 0.044725, 0.044475, 0.0441, 0.04435, 0.044475, 0.0446, 0.045725, 0.046967]

tenors = [Period('6m'), Period('9m'), Period('1Y'), Period('2Y'), Period('3Y'), Period('4Y'), Period('5Y'), Period('7y'), Period('10y')]

## 构造收益率曲线
---------------------------

In [ ]:
bid_quotes = [QuoteHandle(SimpleQuote(r)) for r in bid_rates]
mid_quotes = [QuoteHandle(SimpleQuote(r)) for r in mid_rates]
offer_quotes = [QuoteHandle(SimpleQuote(r)) for r in offer_rates]

shibor_index = Shibor(Period(3, Months))

In [ ]:
bid_instruments = [ShiborSwapRateHelper(r, t, 3 * Months, shibor_index) for r, t in zip(bid_quotes, tenors)]
mid_instruments = [ShiborSwapRateHelper(r, t, 3 * Months, shibor_index) for r, t in zip(mid_quotes, tenors)]
offer_instruments = [ShiborSwapRateHelper(r, t, 3 * Months, shibor_index) for r, t in zip(offer_quotes, tenors)]

In [ ]:
calendar = China(China.IB)
today = calendar.adjust(Date.todaysDate())

Settings.instance().evaluationDate = today

In [ ]:
bid_curve = PiecewiseLinearZero(today, bid_instruments, Actual365Fixed())
mid_curve = PiecewiseLinearZero(today, mid_instruments, Actual365Fixed())
offer_curve = PiecewiseLinearZero(today, offer_instruments, Actual365Fixed())

## 获取收益率曲线数据
-----------------------

In [ ]:
%%time
pillars = [today + i for i in range(3660)]

bid_zero_rates = [bid_curve.zeroRate(d, Actual360(), Continuous, Annual, True).rate() * 100. for d in pillars]
mid_zero_rates = [mid_curve.zeroRate(d, Actual360(), Continuous, Annual, True).rate() * 100. for d in pillars]
offer_zero_rates = [offer_curve.zeroRate(d, Actual360(), Continuous, Annual, True).rate() * 100. for d in pillars]

py_dates = [dt.datetime(d.year(), d.month(), d.dayOfMonth()) for d in pillars]

In [ ]:
plt.figure(figsize=(12, 6))
plt.title("Chinese Shibor/3M Swap Zero Rate curve", fontsize=16)
plt.plot(py_dates, bid_zero_rates)
plt.plot(py_dates, mid_zero_rates)
plt.plot(py_dates, offer_zero_rates)
plt.legend(['bid', 'mid', 'offer'])

In [ ]:
%%time
bid_df = [bid_curve.discount(d, True) for d in pillars]
mid_df = [mid_curve.discount(d, True) for d in pillars]
offer_df = [offer_curve.discount(d, True) for d in pillars]

In [ ]:
plt.figure(figsize=(12, 6))
plt.title("Chinese Shibor/3M Swap Discout Factor curve", fontsize=16)
plt.plot(py_dates, bid_df)
plt.plot(py_dates, mid_df)
plt.plot(py_dates, offer_df)
plt.legend(['bid', 'mid', 'offer'])

In [ ]:
%%time
bid_forward = [bid_curve.forwardRate(d, d, Actual360(), Continuous, Annual, True).rate() * 100. for d in pillars]
mid_forward = [mid_curve.forwardRate(d, d, Actual360(), Continuous, Annual, True).rate() * 100. for d in pillars]
offer_forward = [offer_curve.forwardRate(d, d, Actual360(), Continuous, Annual, True).rate() * 100. for d in pillars]

In [ ]:
plt.figure(figsize=(12, 6))
plt.title("Chinese Shibor/3M Swap Forward Rate curve", fontsize=16)
plt.plot(py_dates, bid_forward)
plt.plot(py_dates, mid_forward)
plt.plot(py_dates, offer_forward)
plt.legend(['bid', 'mid', 'offer'])